<a href="https://colab.research.google.com/github/vladmirkulov13/HPCLabs/blob/main/FirstLab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pycuda

     |████████████████████████████████| 1.7 MB 12.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 75 kB 4.5 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
  Created wheel for pycuda: filename=pycuda-2021.1-cp37-cp37m-linux_x86_64.whl size=627882 sha256=c239ca045d0f2dab6eb038e735c0736f2f94cb1be75fd5fc77e2d585efaf2e8b
  Stored in directory: /root/.cache/pip/wheels/c4/ef/49/dc6a5feb8d980b37c83d465ecab24949a6aa19458522a9e001
  Created wheel for pytools: filename=pytools-2021.2.8-py2.py3-none-any.whl size=60725 sha256=b862a7beb4ea4c7f9ab9b18ae08e8e6e3ed106fb0e842f0f81b353604a9d9b8f
  Stored in directory: /root/.cache/pip/wheels/bd/2d/ef/0127a17bafa44971f11d05d0e38d7947144cf9e33313bf12a7
Successfully built pycuda pytools


In [3]:
import pycuda.autoinit
from pycuda.tools import make_default_context
make_default_context().get_device().name()

'Tesla K80'

In [10]:
import numpy as np
from numpy import linalg as la
from pycuda import driver, compiler, gpuarray, tools
from time import time

for i in range(0, 5):
  
  MATRIX_SIZE = 128*2**i
  print("Размер матрицы: ", MATRIX_SIZE, "x", MATRIX_SIZE)

  def matmul(a_gpu,b_gpu,MATRIX_SIZE=MATRIX_SIZE):
    kernel_code_template = """
    __global__ void MatrixMulKernel(float *A, float *B, float *C)
    {

      const uint wA = %(MATRIX_SIZE)s;
      const uint wB = %(MATRIX_SIZE)s;

      const uint bx = blockIdx.x;
      const uint by = blockIdx.y;

      const uint tx = threadIdx.x;
      const uint ty = threadIdx.y;

      const uint aBegin = wA * %(BLOCK_SIZE)s * by;
      const uint aEnd = aBegin + wA - 1;
      const uint aStep = %(BLOCK_SIZE)s;

      const uint bBegin = %(BLOCK_SIZE)s * bx;
      const uint bStep = %(BLOCK_SIZE)s * wB;

      float Csub = 0;
      for (int a = aBegin, b = bBegin;
           a <= aEnd;
           a += aStep, b += bStep)
        {
          __shared__ float As[%(BLOCK_SIZE)s][%(BLOCK_SIZE)s];
          __shared__ float Bs[%(BLOCK_SIZE)s][%(BLOCK_SIZE)s];

          As[ty][tx] = A[a + wA * ty + tx];
          Bs[ty][tx] = B[b + wB * ty + tx];

          __syncthreads();

          for (int k = 0; k < %(BLOCK_SIZE)s; ++k)
            Csub += As[ty][k] * Bs[k][tx];

          __syncthreads();
        }

      const uint c = wB * %(BLOCK_SIZE)s * by + %(BLOCK_SIZE)s * bx;
      C[c + wB * ty + tx] = Csub;

    }
    """

    TILE_SIZE = 32
    BLOCK_SIZE = TILE_SIZE

    kernel_code = kernel_code_template % {
        'MATRIX_SIZE': MATRIX_SIZE,
        'BLOCK_SIZE': BLOCK_SIZE,
        }

    mod = compiler.SourceModule(kernel_code)
    c_gpu = gpuarray.empty((MATRIX_SIZE, MATRIX_SIZE), np.float32)

    matrixmul = mod.get_function("MatrixMulKernel")

    matrixmul(
        a_gpu, b_gpu,
        c_gpu,
        grid = (MATRIX_SIZE // TILE_SIZE, MATRIX_SIZE // TILE_SIZE),
        block = (TILE_SIZE, TILE_SIZE, 1),
        )

    return c_gpu

  a_cpu = np.random.randn(MATRIX_SIZE, MATRIX_SIZE).astype(np.float32)
  b_cpu = np.random.randn(MATRIX_SIZE, MATRIX_SIZE).astype(np.float32)

  st = time()
  c_cpu = np.dot(a_cpu, b_cpu)
  cpu_time = time() - st

  st = time()
  a_gpu = gpuarray.to_gpu(a_cpu)
  b_gpu = gpuarray.to_gpu(b_cpu)
  c_gpu = matmul(a_gpu,b_gpu).get()
  gpu_time = time() - st

  print("Время выполнения перемножения на CPU: {:.6f}".format(cpu_time))
  print("Время выполнения перемножения на GPU: {:.6f}".format(gpu_time))
  print("Ускорение: {:.3f}".format(cpu_time/gpu_time)) 
  print("Корректность вычислений: {:.6f}".format(np.max(c_cpu - c_gpu)))
  print()
  print("-" * 50)

Размер матрицы:  128 x 128
Время выполнения перемножения на CPU: 0.000955
Время выполнения перемножения на GPU: 0.009115
Ускорение: 0.105
Корректность вычислений: 0.000013

--------------------------------------------------
Размер матрицы:  256 x 256
Время выполнения перемножения на CPU: 0.000763
Время выполнения перемножения на GPU: 0.002154
Ускорение: 0.354
Корректность вычислений: 0.000031

--------------------------------------------------
Размер матрицы:  512 x 512
Время выполнения перемножения на CPU: 0.004732
Время выполнения перемножения на GPU: 0.005198
Ускорение: 0.910
Корректность вычислений: 0.000076

--------------------------------------------------
Размер матрицы:  1024 x 1024
Время выполнения перемножения на CPU: 0.060270
Время выполнения перемножения на GPU: 0.015806
Ускорение: 3.813
Корректность вычислений: 0.000198

--------------------------------------------------
Размер матрицы:  2048 x 2048
Время выполнения перемножения на CPU: 0.251441
Время выполнения перемноже